In [29]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split # train test비율에 맞게 짜르기

import argparse
import os
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')


import easydict

In [22]:
def f1_loss(y_true:torch.Tensor, y_pred:torch.Tensor, is_training=False) -> torch.Tensor:

    assert y_true.ndim == 1
    assert y_pred.ndim == 1 or y_pred.ndim == 2
    
    if y_pred.ndim == 2:
        y_pred = y_pred.argmax(dim=1)
        
    
    tp = (y_true * y_pred).sum().to(torch.float32)
    tn = ((1 - y_true) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_true) * y_pred).sum().to(torch.float32)
    fn = (y_true * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    f1.requires_grad = is_training
    return f1

In [73]:
class LogisticRegression(nn.Module):

    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(32,16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )        

    def forward(self, x):
        return self.linear(x)

In [74]:
model=LogisticRegression()

In [28]:
torch.manual_seed(42)


In [92]:
train = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\train.csv')
test = pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\jobcare_recommend\\JobCare_data\\test.csv')


train = train.drop(['id', 'contents_open_dt'], axis=1) 
test = test.drop(['id', 'contents_open_dt'], axis=1)

print(train.head())
train['d_l_match_yn']=train['d_l_match_yn'].replace([True,False],[1,0])
train['d_m_match_yn']=train['d_m_match_yn'].replace([True,False],[1,0])
train['d_s_match_yn']=train['d_s_match_yn'].replace([True,False],[1,0])
train['h_l_match_yn']=train['h_l_match_yn'].replace([True,False],[1,0])
train['h_m_match_yn']=train['h_m_match_yn'].replace([True,False],[1,0])
train['h_s_match_yn']=train['h_s_match_yn'].replace([True,False],[1,0])

test['d_l_match_yn']=test['d_l_match_yn'].replace([True,False],[1,0])
test['d_m_match_yn']=test['d_m_match_yn'].replace([True,False],[1,0])
test['d_s_match_yn']=test['d_s_match_yn'].replace([True,False],[1,0])
test['h_l_match_yn']=test['h_l_match_yn'].replace([True,False],[1,0])
test['h_m_match_yn']=test['h_m_match_yn'].replace([True,False],[1,0])
test['h_s_match_yn']=test['h_s_match_yn'].replace([True,False],[1,0])

x = train.iloc[0:1000, :-1]
y = train.iloc[0:1000, -1]



x= x.to_numpy()
print(x[0])
print(y[0])
x_train = torch.FloatTensor(x)
y_train = torch.FloatTensor(y)

   d_l_match_yn  d_m_match_yn  d_s_match_yn  h_l_match_yn  h_m_match_yn  \
0          True          True          True         False         False   
1         False         False         False          True          True   
2         False         False         False          True         False   
3         False         False         False          True         False   
4          True          True          True         False         False   

   h_s_match_yn  person_attribute_a  person_attribute_a_1  person_attribute_b  \
0         False                   1                     4                   3   
1         False                   1                     3                   4   
2         False                   2                     0                   3   
3         False                   2                     0                   2   
4         False                   1                     3                   4   

   person_prefer_c  ...  contents_attribute_c  contents_attrib

In [64]:
print(x_train.shape)
print(y_train.shape)



torch.Size([1000, 32])
torch.Size([1000])


In [102]:
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    hypothesis=hypothesis.view([1000])
    print(hypothesis.size())
    print(y_train.size())
    # print(hypothesis)
    prediction = hypothesis >= torch.FloatTensor([0.5])
    prediction= prediction.float()
    print(y_train)
    print(prediction)
    # break


    # cost 계산
    cost = F.binary_cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == y_train # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

torch.Size([1000])
torch.Size([1000])
tensor([1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
        1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
  

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [67]:
print(W)
print(b)

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], requires_grad=True)
tensor([nan], requires_grad=True)


In [52]:
import torch
from torch.utils.data import DataLoader, TensorDataset

sample=len(train)
# Create the dataset with N_SAMPLES samples
D_in, H, D_out = 32, 16, 1 #샘플 개수, 인풋값 , 중간값, 결과값 개수

x_train = torch.FloatTensor(x)
y_train = torch.FloatTensor(y)

# Define the batch size and the number of epochs
BATCH_SIZE = 128
N_EPOCHS = 30

# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Define model, loss and optimizer
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)

# Loop over epochs
for epoch in range(N_EPOCHS):
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):

        y_batch_pred = model(x_batch)

        loss = loss_fn(y_batch_pred, y_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Every 100 batches, print the loss for this batch
        # as well as the number of examples processed so far 
        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")

Epoch 1
-------------------------------
loss: 1005628096512.000000  [  128/501951]
loss: 424864579584.000000  [12928/501951]
loss: 100933312512.000000  [25728/501951]
loss: 144303095808.000000  [38528/501951]
loss: 24015765504.000000  [51328/501951]
loss: 13416057856.000000  [64128/501951]
loss: 14653132800.000000  [76928/501951]
loss: 7364025344.000000  [89728/501951]
loss: 7964484608.000000  [102528/501951]
loss: 3720664064.000000  [115328/501951]
loss: 4370186752.000000  [128128/501951]
loss: 4594412544.000000  [140928/501951]
loss: 2557721600.000000  [153728/501951]
loss: 1863687040.000000  [166528/501951]
loss: 1154097664.000000  [179328/501951]
loss: 1370611072.000000  [192128/501951]
loss: 594336512.000000  [204928/501951]
loss: 689816960.000000  [217728/501951]
loss: 539333056.000000  [230528/501951]
loss: 1538718336.000000  [243328/501951]
loss: 1023725696.000000  [256128/501951]
loss: 785935488.000000  [268928/501951]
loss: 803020288.000000  [281728/501951]
loss: 719183872.00

KeyboardInterrupt: 